# Notebook for a space-time discretization

In this notebook we are treating following problem 
$$
u_t - \Delta u + \mathbf{w} \cdot \nabla u = f \quad \text{in } \Omega  = [0,1] \times [0,T].
$$
with the initial/boundary conditions
$$
u(x,0) = \sin(\pi x), \quad u(0,t) = u(1,t)=0.
$$
The weakformulation of the problem in the CG setting is

Find a function $u_0 \in H^1_0(\Omega)$ , such that  for all  $v \in H^1_0(\Omega) $:

$$
\int_{\Omega} (u_t)_0 v \,dx \,dt
+ \int_{\Omega} \nabla u_0 \cdot \nabla v \,dx \,dt
+ \int_{\Omega} (\mathbf{w} \cdot \nabla u_0) v \,dx \,dt
= \int_{\Omega} f v \,dx \,dt
- \int_{\Omega} u_D v \,dx \,dt.
$$

where we used the decomposition of $u = u_D + u_0$

For $w = 0$ (the heatequation), we have for the chosen initial/boundary conditions following analytical solution 

$u(x,t)= \sin(\pi x) e^{-t \pi^2}$

In [3]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *

p = -1
T =8 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.75))
Draw(mesh)

for el in mesh.Elements():
        for vi in el.vertices:
            # Hole den Punkt des Vertex
            print(mesh[vi].point)
            # Wende die Deformation an und speichere den neuen Punkt

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

(0.0, 0.0)
(1.0, 0.0)
(1.0, 0.7272727272727272)
(1.0, 0.7272727272727272)
(1.0, 1.4545454545454544)
(-1.3359783263425669e-15, 0.7272727272727284)
(1.0000000000000002, 2.1818181818181817)
(-1.0687826610740532e-15, 2.1818181818181843)
(-1.20238049370831e-15, 1.4545454545454568)
(1.0000000000000002, 2.1818181818181817)
(1.0000000000000002, 2.909090909090909)
(-1.0687826610740532e-15, 2.1818181818181843)
(1.0000000000000002, 3.636363636363636)
(-8.015869958055402e-16, 3.6363636363636367)
(-9.351848284397967e-16, 2.909090909090911)
(1.0000000000000002, 3.636363636363636)
(1.0000000000000002, 4.363636363636363)
(-8.015869958055402e-16, 3.6363636363636367)
(1.0000000000000002, 5.090909090909089)
(-5.343913305370269e-16, 5.090909090909091)
(-6.679891631712834e-16, 4.363636363636364)
(1.0000000000000002, 5.090909090909089)
(1.0000000000000002, 5.818181818181816)
(-5.343913305370269e-16, 5.090909090909091)
(1.0000000000000004, 6.545454545454543)
(-2.6719566526851344e-16, 6.545454545454545)
(-4.0

In [2]:
deformation = GridFunction(H1(mesh,order=1,dim=mesh.dim))
deformation.Set(CF((x,y/8))-CF((x,y)))
#Draw(deformation,mesh)
mesh.SetDeformation(deformation)
Draw(mesh)
#for el in mesh.Elements():
 #       for vi in el.vertices:
            #x = tuple(deformation(mesh[vi].point[0],mesh[vi].point[1]))
            #print(tuple((x[0]+mesh[vi].point[0],x[1]+mesh[vi].point[1])))


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:
Draw(y,mesh,deformation=deformation)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.1))

In [1]:
def abs(u):
        return IfPos(u[1],u,-u)

def addtuple(t1,t2):
        return tuple(a + b for a, b in zip(t1, t2))

def maximumy(mesh,deformation):
        # Iteriere über alle Dreiecke im Mesh
        max_y_difference = 0
        max_x_difference = 0
        for el in mesh.Elements():
                vertices = [addtuple(tuple(deformation(mesh[vi].point[0],mesh[vi].point[1])),tuple(mesh[vi].point)) for vi in el.vertices]  # Eckpunkte des Dreiecks
                y_coords = [p[1] for p in vertices]  # Nur die y-Koordinaten extrahieren
                x_coords = [p[0] for p in vertices]
                
                y_difference = max(y_coords) - min(y_coords)  # Höchster - niedrigster Wert
                max_y_difference = max(max_y_difference, y_difference)  # Maximale Differenz speichern
                x_difference = max(x_coords) - min(x_coords)
                max_x_difference = max(max_x_difference, x_difference)

        return max_y_difference,max_x_difference







In [7]:
meshes =[]
for p in range(-3,2):
        mesh1 =[]
        for i in range(-3,2):
                L = 2**i
                T = 2**(p) # length of the time interval
                shape = Rectangle(L,T).Face()
                shape.edges.Min(X).name="left"
                shape.edges.Max(X).name="right"
                shape.edges.Min(Y).name="bottom"
                shape.edges.Max(Y).name="top"
                mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.025))
                deformation = GridFunction(H1(mesh,order=1,dim=mesh.dim))
                deformation.Set(CF((2**-i*x,2**(-p)*y))-CF((x,y)))
                mesh.SetDeformation(deformation)
                #Draw(sin(y),mesh,deformation=deformation)
                #print(f"p = {p}" + f" i = {i}")       
                may = maximumy(mesh,deformation=deformation)
                mesh1.append((mesh,may))
        meshes.append(mesh1)

from tabulate import tabulate
import pandas as pd
import numpy as np
df = pd.DataFrame(meshes)
# Extrahiere die ersten und zweiten Werte der Tupel in separate Listen
first_values = [[item[1][0] for item in row] for row in df.values]
second_values = [[item[1][1] for item in row] for row in df.values]
average_row_first_values = [np.mean(row) for row in first_values]
average_column_second_values = [np.mean(col) for col in zip(*second_values)]
print(average_row_first_values,average_column_second_values)
#print(df)
print(tabulate(meshes, headers=["p = -3","p = -2","p = -1","p = 0"], tablefmt="grid"))

[0.28625704151917286, 0.13437486425793072, 0.06793728230500644, 0.03352154055824248, 0.01668083513329808] [0.24793609870779704, 0.12580978412836177, 0.06473510995895201, 0.03182366368775117, 0.016408018386266165]
+---------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------+
|                                                                                             | p = -3                                                                                      | p = -2                                                                                     | p = -1                         